사업보고서로부터 전체 기업의 주요 재무지표를 수집하여 가공해보자<br>
부채비율, 영업이익증가율, 매출액증가율, 매출액 상태, ROA, ROE 계산해보기.

In [4]:
import OpenDartReader
import os
from dotenv import load_dotenv

load_dotenv()
api = os.getenv("API_KEY")
dart = OpenDartReader(api)

In [5]:
import pandas as pd

base_dir = os.path.dirname(os.getcwd())
file_path = os.path.join(base_dir, "data", "종목정보.txt")

stock_list = pd.read_csv(file_path, encoding = "euc-kr", sep = "\t",
                         usecols=["Name", "Symbol", "Market"], dtype=str)

report = dart.finstate("삼성전자", 2020)
display(report[["fs_nm", "account_nm", "thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]])
# this term(당기), former term(전기), before former term(전전기)

,fs_nm,account_nm,thstrm_amount,frmtrm_amount,bfefrmtrm_amount
0,연결재무제표,유동자산,"198,215,579,000,000","181,385,260,000,000","174,697,424,000,000"
1,연결재무제표,비유동자산,"180,020,139,000,000","171,179,237,000,000","164,659,820,000,000"
2,연결재무제표,자산총계,"378,235,718,000,000","352,564,497,000,000","339,357,244,000,000"
3,연결재무제표,유동부채,"75,604,351,000,000","63,782,764,000,000","69,081,510,000,000"
4,연결재무제표,비유동부채,"26,683,351,000,000","25,901,312,000,000","22,522,557,000,000"
5,연결재무제표,부채총계,"102,287,702,000,000","89,684,076,000,000","91,604,067,000,000"
6,연결재무제표,자본금,"897,514,000,000","897,514,000,000","897,514,000,000"
7,연결재무제표,이익잉여금,"271,068,211,000,000","254,582,894,000,000","242,698,956,000,000"
8,연결재무제표,자본총계,"275,948,016,000,000","262,880,421,000,000","247,753,177,000,000"
9,연결재무제표,매출액,"236,806,988,000,000","230,400,881,000,000","243,771,415,000,000"


In [6]:
import numpy as np
import pandas as pd

def find_fins_ind_list(stock_code, stock_name, year, ind_list, verbose=False):
    try: # 데이터 가져오기
        report = None
        report = dart.finstate(stock_code, year)
        if isinstance(report, pd.DataFrame) and report.empty:
            if verbose:
                print("빈 DataFrame — 데이터 없음")
        elif isinstance(report, dict) and report.get("status") == "013":
            if verbose:
                print("API 응답 status 013 — 데이터 없음")
        elif not isinstance(report, pd.DataFrame):
            raise ValueError(f"report 타입 문제 - type: {type(report)}")
        elif "account_nm" not in report.columns:
            raise ValueError(f"account_nm 없음 - columns: {list(report.columns)}")
            
    except Exception as e:
        if verbose:
            print(f"예외 발생: {e}")
        report = None

    if report is None: 
        # 리포트가 없으면 당기, 전기, 전전기 값을 모두 제거
        data = [[stock_name, year] + [np.nan] * len(ind_list)]
        data.append([stock_name, year - 1] + [np.nan] * len(ind_list))
        data.append([stock_name, year - 2] + [np.nan] * len(ind_list))

    else: 
        report = report[report["account_nm"].isin(ind_list)] # 관련 지표로 필터링
        if sum(report["fs_nm"] == "연결재무제표") > 0:
            # 연결재무제표 데이터가 있으면 연결재무제표 사용
            report = report.loc[report["fs_nm"] == "연결재무제표"]
        else:
            # 연결재무제표 데이터가 없으면 일반재무제표 사용
            report = report.loc[report["fs_nm"] == "재무제표"]
        data = []
        for y, c in zip([year, year - 1, year - 2],
                        ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]):
            record = [stock_name, y]
            for ind in ind_list:
                # acccount_nm이 indic인 행의 c 칼럼 값을 가져옴
                if sum(report["account_nm"] == ind) > 0:
                    value = report.loc[report["account_nm"] == ind, c].iloc[0]
                else:
                    value = np.nan
                record.append(value)
            data.append(record)

    return pd.DataFrame(data, columns=["기업", "연도"] + ind_list)

In [7]:
# 미리 계정명 출력해보기
report = dart.finstate("096760", 2020)
print(report["account_nm"].unique())

['유동자산' '비유동자산' '자산총계' '유동부채' '비유동부채' '부채총계' '자본금' '이익잉여금' '자본총계' '매출액'
 '영업이익' '법인세차감전 순이익' '당기순이익' '당기순이익(손실)']


In [8]:
ind_list = ["자산총계", "부채총계", "자본총계", "매출액", "영업이익", "당기순이익"]
display(find_fins_ind_list("005930", "삼성전자", 2020, ind_list))

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
0,삼성전자,2020,"378,235,718,000,000","102,287,702,000,000","275,948,016,000,000","236,806,988,000,000","35,993,876,000,000","26,407,832,000,000"
1,삼성전자,2019,"352,564,497,000,000","89,684,076,000,000","262,880,421,000,000","230,400,881,000,000","27,768,509,000,000","21,738,865,000,000"
2,삼성전자,2018,"339,357,244,000,000","91,604,067,000,000","247,753,177,000,000","243,771,415,000,000","58,886,669,000,000","44,344,857,000,000"


In [9]:
print(stock_list.columns.tolist())

['Symbol', 'Market', 'Name']


In [10]:
import time
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

data = pd.DataFrame()
errors = []

kospi_df = stock_list[stock_list['Market'] == 'KOSPI']
iter_data = kospi_df[['Symbol', 'Name']].values

pbar = tqdm(range(len(iter_data)), desc="코스피 대상 작업 진행중", leave=True)

for i in pbar:
    code, name = iter_data[i]
    for year in [2015, 2018, 2020]:
        pbar.set_postfix_str(f"{name}, {year}")
        try:
            result = find_fins_ind_list(code, name, year, ind_list, verbose=False)
            data = pd.concat([data, result], axis=0, ignore_index=True)
            time.sleep(0.1)
        except Exception as e:
            # print 대신 tqdm.write() 사용하되 notebook에서는 깔끔하게 출력되게끔 처리
            tqdm.write(f"예외 발생 - {name} ({code}), {year}: {e}")
            errors.append((code, name, year, str(e)))
        time.sleep(0.05)



# 작업 끝나고 에러 요약
error_df = pd.DataFrame(errors, columns=["종목코드", "종목명", "연도", "에러내용"])
pd.set_option('display.max_colwidth', None)
display(error_df)


코스피 대상 작업 진행중:   0%|          | 0/823 [00:00<?, ?it/s]

{'status': '013', 'message': '조회된 데이타가 없습니다.'}

예외 발생 - BGF리테일 (282330), 2015: 'account_nm'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

예외 발생 - BNK금융지주 (138930), 2015: 'account_nm'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

예외 발생 - BNK금융지주 (138930), 2018: 'account_nm'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

예외 발생 - BNK금융지주 (138930), 2020: 'account_nm'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

예외 발생 - DB금융투자 (016610), 2015: 'account_nm'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

예외 발생 - DB금융투자 (016610), 2018: 'account_nm'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

예외 발생 - DB금융투자 (016610), 2020: 'account_nm'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

예외 발생 - DB손해보험 (005830), 2015: 'account_nm'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

예외 발생 - DB손해보험 (005830), 2018: 'account_nm'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

예외 발생 - DB손해보험 (005830), 2020: 'account_nm'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

예외 발생 - DGB금융지주 (139130), 201

,종목코드,종목명,연도,에러내용
0,282330,BGF리테일,2015,'account_nm'
1,138930,BNK금융지주,2015,'account_nm'
2,138930,BNK금융지주,2018,'account_nm'
3,138930,BNK금융지주,2020,'account_nm'
4,016610,DB금융투자,2015,'account_nm'
...,...,...,...,...
316,298020,효성티앤씨,2015,'account_nm'
317,298000,효성화학,2015,'account_nm'
318,000540,흥국화재,2015,'account_nm'
319,000540,흥국화재,2018,'account_nm'


In [11]:
data.drop_duplicates(inplace = True) # 2018년 행 중복되므로 삭제
data.sort_values(by = ['기업', '연도'], inplace = True) # 먼저 기업 이름, 이후 연도 기준으로 오름차순 정렬. 원본 data에 바로 반영.
display(data.head(10))

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
2,AJ네트웍스,2013,"1,250,258,682,892","1,035,947,421,718","214,311,261,174","757,134,207,083","70,537,595,517","18,131,663,759"
1,AJ네트웍스,2014,"1,437,917,070,538","1,154,639,967,755","283,277,102,783","1,011,150,088,928","76,977,721,440","28,058,455,257"
0,AJ네트웍스,2015,"1,725,200,197,495","1,327,188,368,579","398,011,828,916","1,055,580,857,057","74,258,957,322","28,991,066,418"
5,AJ네트웍스,2016,"2,081,316,853,429","1,653,831,151,606","427,485,701,823","1,253,876,534,610","61,859,364,996","17,079,187,443"
4,AJ네트웍스,2017,"2,354,233,734,500","1,907,060,853,223","447,172,881,277","843,940,516,906","22,438,377,217","20,647,179,903"
3,AJ네트웍스,2018,"2,575,139,461,359","2,083,331,488,612","491,807,972,747","1,056,691,254,095","-21,281,686,850","37,268,842,438"
8,AJ네트웍스,2018,"2,575,139,461,359","2,083,331,488,612","491,807,972,747","1,045,526,121,935","-21,236,052,082","37,268,842,438"
7,AJ네트웍스,2019,"1,803,255,225,986","1,455,914,070,366","347,341,155,620","1,000,259,401,598","15,649,322,684","42,128,594,358"
6,AJ네트웍스,2020,"1,588,170,350,000","1,300,162,539,894","288,007,810,106","1,017,050,375,182","22,952,778,020","-3,316,812,122"
11,AK홀딩스,2013,"2,374,994,471,896","1,739,973,013,119","635,021,458,777","2,240,972,020,725","108,947,071,649","108,312,125,916"


In [12]:
# 칼럼 값을 숫자로 변환하는 함수
def convert_str_to_float(value):
    if type(value) == float: # nan의 자료형도 float임
        return value
    elif value == "-": # 음수에도 하이픈이 있기에, value가 하이픈일 때에만 변환한다.
        return 0
    else: 
        return float(value.replace(",", ""))

for ind in ind_list:
    data[ind] = data[ind].apply(convert_str_to_float) # 기업과 연도를 제외한 칼럼인 ind_list를 순회하며 함수 적용하기.

In [13]:
display(data.head(10))

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익
2,AJ네트웍스,2013,1.250259e+12,1.035947e+12,2.143113e+11,7.571342e+11,7.053760e+10,1.813166e+10
1,AJ네트웍스,2014,1.437917e+12,1.154640e+12,2.832771e+11,1.011150e+12,7.697772e+10,2.805846e+10
0,AJ네트웍스,2015,1.725200e+12,1.327188e+12,3.980118e+11,1.055581e+12,7.425896e+10,2.899107e+10
5,AJ네트웍스,2016,2.081317e+12,1.653831e+12,4.274857e+11,1.253877e+12,6.185936e+10,1.707919e+10
4,AJ네트웍스,2017,2.354234e+12,1.907061e+12,4.471729e+11,8.439405e+11,2.243838e+10,2.064718e+10
3,AJ네트웍스,2018,2.575139e+12,2.083331e+12,4.918080e+11,1.056691e+12,-2.128169e+10,3.726884e+10
8,AJ네트웍스,2018,2.575139e+12,2.083331e+12,4.918080e+11,1.045526e+12,-2.123605e+10,3.726884e+10
7,AJ네트웍스,2019,1.803255e+12,1.455914e+12,3.473412e+11,1.000259e+12,1.564932e+10,4.212859e+10
6,AJ네트웍스,2020,1.588170e+12,1.300163e+12,2.880078e+11,1.017050e+12,2.295278e+10,-3.316812e+09
11,AK홀딩스,2013,2.374994e+12,1.739973e+12,6.350215e+11,2.240972e+12,1.089471e+11,1.083121e+11


# 주요 지표 계산
부채비율 = 자본총계 / 자산총계<br>
매출액 증가율 = (당기-전기)매출액 / 전기 매출액<br>
영업이익 증가율 = (당기-전기)영업이익 / 전기 영업이익<br>
당기순이익 증가율 = (당기-전기)당기순이익 / 전기 영업이익<br>
(단순 재무지표로는 전기 순이익, 본업의 경영 효율성이나 투자 성과 평가를 보기 위해서는 전기 영업이익을 분모로 사용.)<br>
ROA = 당기순이익 / 자산총계 <br>
ROE = 당기순이익 / 평균 자기자본

In [14]:
# 부채비율 = 자본총계 / 자산총계 * 100%
data["부채비율"] = data["자본총계"] / data["자산총계"] * 100
display(data["부채비율"].head())  # DataFrame에 새로운 컬럼(또는 기존 컬럼을 덮어쓰기)을 생성하는 문법임.
# pandas.Series는 인덱스를 가진 1차열 배열.

display(data.head(10))

2    17.141354
1    19.700517
0    23.070472
5    20.539194
4    18.994413
Name: 부채비율, dtype: float64

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,부채비율
2,AJ네트웍스,2013,1.250259e+12,1.035947e+12,2.143113e+11,7.571342e+11,7.053760e+10,1.813166e+10,17.141354
1,AJ네트웍스,2014,1.437917e+12,1.154640e+12,2.832771e+11,1.011150e+12,7.697772e+10,2.805846e+10,19.700517
0,AJ네트웍스,2015,1.725200e+12,1.327188e+12,3.980118e+11,1.055581e+12,7.425896e+10,2.899107e+10,23.070472
5,AJ네트웍스,2016,2.081317e+12,1.653831e+12,4.274857e+11,1.253877e+12,6.185936e+10,1.707919e+10,20.539194
4,AJ네트웍스,2017,2.354234e+12,1.907061e+12,4.471729e+11,8.439405e+11,2.243838e+10,2.064718e+10,18.994413
3,AJ네트웍스,2018,2.575139e+12,2.083331e+12,4.918080e+11,1.056691e+12,-2.128169e+10,3.726884e+10,19.098304
8,AJ네트웍스,2018,2.575139e+12,2.083331e+12,4.918080e+11,1.045526e+12,-2.123605e+10,3.726884e+10,19.098304
7,AJ네트웍스,2019,1.803255e+12,1.455914e+12,3.473412e+11,1.000259e+12,1.564932e+10,4.212859e+10,19.261897
6,AJ네트웍스,2020,1.588170e+12,1.300163e+12,2.880078e+11,1.017050e+12,2.295278e+10,-3.316812e+09,18.134567
11,AK홀딩스,2013,2.374994e+12,1.739973e+12,6.350215e+11,2.240972e+12,1.089471e+11,1.083121e+11,26.737808


In [15]:
data["매출액증가율"] = data["매출액"].diff() / data["매출액"].shift(1) * 100
# diff(n) 메서드는 시리즈의 각 요소와 n번째 전 값의 차이를 계산하며, n의 기본값은 1임. shift(1)은 전기로 한 칸 이동.
data.loc[data["연도"] == 2013, "매출액증가율"] = np.nan
# data에서 가장 이른 연도가 2013년임. 전기 대비 증가율을 알 수 없으므로 모두 nan으로 처리함.
data["영업이익증가율"] = data["영업이익"].diff() / data["영업이익"].shift(1) * 100
data.loc[data["연도"] == 2013, "영업이익증가율"] = np.nan
data["당기순이익증가율"] = data["당기순이익"].diff() / data["영업이익"].shift(1) * 100
data.loc[data["연도"] == 2013, "당기순이익증가율"] = np.nan

In [20]:
# 증가율은 전기와 당기 값이 모두 양수일 때에만 사용하는 것이 바람직하다.
# 흑자, 적자 상태를 나타내는 함수 정의
def add_state(data, col):
    data[col + "_상태"] = pd.Series(dtype="object")

    value = data[col].to_numpy()
    cur_value = value[1:]
    pre_value = value[:-1]
    # 흑자지속
    cond1 = (cur_value > 0) & (pre_value > 0)
    cond1 = np.insert(cond1, 0, np.nan)  # np.insert(기존 배열, 삽입할 위치, 삽입할 값)
    # 적자지속
    cond2 = (cur_value < 0) & (pre_value < 0)
    cond2 = np.insert(cond2, 0, np.nan)
    # 흑자전환
    cond3 = (cur_value > 0) & (pre_value < 0)
    cond3 = np.insert(cond3, 0, np.nan)
    # 적자전환
    cond4 = (cur_value < 0) & (pre_value > 0)
    cond4 = np.insert(cond4, 0, np.nan)

    # 조건에 따른 변환
    data.loc[cond1, col + "_상태"] = "흑자지속"
    data.loc[cond2, col + "_상태"] = "적자지속"
    data.loc[cond3, col + "_상태"] = "흑자전환"
    data.loc[cond4, col + "_상태"] = "적자전환"

In [21]:
add_state(data, "매출액")
add_state(data, "영업이익")
add_state(data, "당기순이익")

In [22]:
# ROA와 ROE
data["ROA"] = data["당기순이익"] / data["자산총계"] * 100
average_equity = data["자본총계"].rolling(2).mean()
data["ROE"] = data["당기순이익"] / average_equity * 100
data.loc[data["연도"] == 2013, "ROE"] = np.nan

In [26]:
import os

base_dir = os.path.dirname(os.getcwd())
file_path = os.path.join(base_dir, "주요재무지표.csv")

display(data)
data.to_csv(file_path, index=False, encoding="euc-kr")

,기업,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,부채비율,매출액증가율,영업이익증가율,당기순이익증가율,매출액_상태,영업이익_상태,당기순이익_상태,ROA,ROE
2,AJ네트웍스,2013,1.250259e+12,1.035947e+12,2.143113e+11,7.571342e+11,7.053760e+10,1.813166e+10,17.141354,NaN,NaN,NaN,적자전환,적자전환,적자전환,1.450233,NaN
1,AJ네트웍스,2014,1.437917e+12,1.154640e+12,2.832771e+11,1.011150e+12,7.697772e+10,2.805846e+10,19.700517,33.549651,9.130062,14.073051,흑자지속,흑자지속,흑자지속,1.951326,11.277778
0,AJ네트웍스,2015,1.725200e+12,1.327188e+12,3.980118e+11,1.055581e+12,7.425896e+10,2.899107e+10,23.070472,4.394082,-3.531884,1.211534,흑자지속,흑자지속,흑자지속,1.680447,8.510652
5,AJ네트웍스,2016,2.081317e+12,1.653831e+12,4.274857e+11,1.253877e+12,6.185936e+10,1.707919e+10,20.539194,18.785456,-16.697773,-16.041000,흑자지속,흑자지속,흑자지속,0.820595,4.137914
4,AJ네트웍스,2017,2.354234e+12,1.907061e+12,4.471729e+11,8.439405e+11,2.243838e+10,2.064718e+10,18.994413,-32.693491,-63.726790,5.767910,흑자지속,흑자지속,흑자지속,0.877023,4.721198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6439,흥아해운,2017,8.734091e+11,7.374264e+11,1.359827e+11,8.364275e+11,-1.299947e+10,-6.199193e+10,15.569191,0.562842,-320.484182,-760.170328,흑자지속,적자전환,적자지속,-7.097697,-37.745872
6438,흥아해운,2018,8.036985e+11,7.305503e+11,7.314817e+10,7.538656e+11,-3.759597e+10,-8.654016e+10,9.101445,-9.870781,189.211595,188.840240,흑자지속,적자지속,적자지속,-10.767739,-82.761706
6443,흥아해운,2018,8.036985e+11,7.305503e+11,7.314817e+10,1.008209e+11,-1.120275e+10,-8.654016e+10,9.101445,-86.626147,-70.202256,-0.000000,흑자지속,적자지속,적자지속,-10.767739,-118.308018
6442,흥아해운,2019,4.309627e+11,4.000433e+11,3.091938e+10,1.021668e+11,-1.236429e+10,-5.135655e+10,7.174491,1.335003,10.368371,-314.062219,흑자지속,적자지속,적자지속,-11.916704,-98.698483


In [109]:
# 재잘대는, verbose
# 디버깅, 로그, 출력 메시지를 자세히 보여줄 것인지의 여부를 결정할 때 쓴다. True나 False를 넣는다.

data = [1,2,3,4,5]
def my_function(data, verbose=True):
    if verbose:
        print(f"데이터 길이: {len(data)}")
        print("처리 시작합니다...")
    
    # 실제 로직
    result= print(data)
    
    if verbose:
        print("처리 완료!")
    
    return result

my_function(data)

데이터 길이: 5
처리 시작합니다...
[1, 2, 3, 4, 5]
처리 완료!
